In [1]:
import os


In [2]:
%pwd

'f:\\end-to-end-deep_learning-project\\research'

In [3]:
os.chdir("../")
%pwd

'f:\\end-to-end-deep_learning-project'

# In the model_trainer_stage, your goal is simple:

⭐ Take the updated base model → Train it → Save the trained model.

No confusion.
No extras.
Here are the only tasks the training stage must do.

✅ 1. Load the Updated Base Model

You previously saved:

base_model.pt → frozen base

base_model_updated.pt → base + new classifier

Training stage ALWAYS loads:

updated_base_model_path     (e.g. base_model_updated.pth)

✅ 2. Load Training Data

Your config already gives you:

training_data = Path(training_data_dir)


Model trainer must:

Use ImageFolder

Apply transforms

Create DataLoader

✅ 3. Set Up Loss & Optimizer

Because you already

froze feature layers

unfroze classifier only

You must define:

CrossEntropyLoss

Adam(model.classifier.parameters(), lr=...)

✅ 4. Run the Training Loop

For each epoch:

iterate through dataloader

forward pass

compute loss

backward pass

optimizer step

accumulate loss

✅ 5. Save the Trained Model

Save as:

trained_model_path (e.g. artifacts/training/model.pth)


This is your final trained model.

# 🌟 What your trainer stage code will logically look like

(no code, just logic)

Load updated_base_model

Load training dataset from folder

Apply transforms (resize, normalize)

Create DataLoader

Move model to device

Define loss + optimizer

For each epoch:

    For each batch:

        Forward pass

        Compute loss

        Backward pass
        
        Update parameters

    Print epoch loss
    
Save trained model


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate:float

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
#from cnnClassifier.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        #training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        training_data_path = self.config.data_ingestion.unzip_dir
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data_path),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config

In [ ]:
import os
import torch
import torch.nn as nn
from pathlib import Path
from torchvision.datasets import ImageFolder # class 
from torch.utils.data import DataLoader
from torch.utils.data import random_split # function
from torchvision import transforms # module



In [ ]:
class Training:

  def __init__(self,config:TrainingConfig):
    
    self.config = config
    self.model = torch.load(self.config.updated_base_model_path)
    self.train_loader = None
    self.val_loader = None
    self.loss_func = None
    self.opt = None
    self.train_data = None
    self.val_data = None
    self.train_loader = None
    self.val_loader = None


  def load_data(self):

    data_path = Path(self.config.training_data)
    img_size = self.config.params_image_size

    transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor()
                                  ])

    dataset = ImageFolder(data_path,transform=transform)
    self.split_dataset(dataset)
    

  
  def split_dataset(self,dataset):

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    self.train_data, self.val_data = random_split(dataset, [train_size, val_size])

    
  def setup_training(self):

    
    batch_size = self.config.params_batch_size
    lr = self.config.params_learning_rate

    self.train_loader = DataLoader(self.train_data, batch_size=batch_size, shuffle=True)
    self.val_loader = DataLoader(self.val_data, batch_size=batch_size)
    self.loss_func = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.model.classifier.parameters(), lr=lr)

  @staticmethod
  def accuracy(out,label):

    preds = torch.argmax(out,dim=1)
    batch_acc= torch.sum(preds == label) / len(label)
    return batch_acc


  def train_model(self, loader=None, opt=None):

    losses = []
    accs = []
    total_samples = 0

    # -------------------------
    # TRAINING mode
    # -------------------------
    if opt is not None:
        self.model.train()

        for xb, yb in loader:
            out = self.model(xb)
            batch_loss = self.loss_func(out, yb)
            batch_acc = self.accuracy(out, yb)

            # update classifier parameters
            opt.zero_grad()
            batch_loss.backward()
            opt.step()

            batch_size = len(xb)
            losses.append(batch_loss.item() * batch_size)
            accs.append(batch_acc.item() * batch_size)
            total_samples += batch_size


    # -------------------------
    # VALIDATION mode
    # -------------------------
    else:
        self.model.eval()

        with torch.no_grad():       # <---- This is all you need
            for xb, yb in loader:
                out = self.model(xb)
                batch_loss = self.loss_func(out, yb)
                batch_acc = self.accuracy(out, yb)

                batch_size = len(xb)
                losses.append(batch_loss.item() * batch_size)
                accs.append(batch_acc.item() * batch_size)
                total_samples += batch_size


    avg_loss = sum(losses) / total_samples
    avg_acc = sum(accs) / total_samples

    return avg_loss, avg_acc


  def fit(self):

    epochs = self.config.params_epochs
    train_losses, train_accs, val_losses, val_accs = [],[],[],[]

    for epoch in range(epochs):
      loss, acc = self.train_model(loader=self.train_loader,opt=self.optimizer)
      train_losses.append(loss)
      train_accs.append(acc)

      val_loss, val_acc = self.train_model(loader=self.val_loader,opt=None)
      val_losses.append(val_loss)
      val_accs.append(val_acc)


      print(f"Epoch {epoch+1} Loss: {loss:.4f} Accuracy : {acc} val_Loss: {val_loss:.4f} val_Accuracy : {val_acc}")

    #return train_losses, train_accs, val_losses, val_accs

  
  def save_model(self):
    """Save full PyTorch model"""
    path = self.config.trained_model_path
    torch.save(self.model, path)

  def run(self):
    self.load_data()
    self.setup_training()
    self.fit()
    self.save_model()


In [ ]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.run()
    
except Exception as e:
    raise e 

[2025-12-12 15:17:16,409: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-12 15:17:16,489: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-12 15:17:16,489: INFO: common: created directory at: artifacts]
[2025-12-12 15:17:16,500: INFO: common: created directory at: artifacts\training]


Epoch 1 Loss: 0.2817 Accuracy : 0.8584269650866476 val_Loss: 0.1732 val_Accuracy : 0.9017857142857143
